###  ADDIS ABABA UNIVERSITY
    
###  ADDIS ABABA INSTITUTE OF TECHNOLOGY
    
####  SCHOOL OF INFORMATION TECHNOLOGY AND ENGINEERING – SITE
    
####  Department of MSC in Artificial Intelligence                                           



### Course Assignment       
>>###### Course:  Distributed Computing for AI
>>###### Name:    Mintesnot Fikir
>>###### Section: Regular
>>###### IDs:   GSR1669/15   
>>###### Submitted to: Beakal Gizachew (PhD)
>>###### Submission date February 2024

# Assignment 5 spark
# Part 1 - Implement and Analyze Word Count
# Part 1 A
For the first part of this homework, you must write a basic word count program that reads in a
single text file, counts the number of occurrences of each word in that text file, and outputs the
result back to a text file as a list of key-value pairs, where the key is the word, and the value is
the number of times the word occurred in the text file.
In addition to basic word counting your code must also do the following:
1. It must be case insensitive (see lower() in Python)
2. It must ignore all punctuation (see, for example, translate() in Python)
3. It must ignore stop words (see filter() in Spark)
4. The output must be sorted by count in descending order (see sortBy() in Spark)
To accomplish this you will use a combination of basic Python, and RDD operations in PySpark


# Part 1 B
Extending Your Application
Once you have your basic word count working for a single text file as described above, you will
now extend it to work for a list of text files, combining the results from each file into a single RDD
of key-value pairs, where the key is the word and the value is the number of times that word
appeared across all passed in text files. The result should only be sorted and written to file once
after all text files have been processed.
<br>





Note: This is different from the in class example where we combined results using join(). See
instead union() in Spark.

## **Analysis**

Answer the following questions based on your WordCount application.
1. In the PySpark REPL, run your basic word count program on a single text file.
a. What are the 25 most common words? Include a screenshot of program output to
back-up your claim.
b. How many stages is execution broken up into? Explain why. Include a screenshot
of the DAG visualization from Spark's WebUI to back-up your claim.
2. In the PySpark REPL, run your extended word count program on all 10 text files.
a. What are the 25 most common words? Include a screenshot of program output to
back-up your claim.
b. How many stages is execution broken up into? Explain why. Include a screenshot
of the DAG visualization from Spark's WebUI to back-up your claim.
3. Your WordCount application should compute the same results as your WordCount
application from Homework #1. Answer the following based on your knowledge of both
MapReduce and Spark:
a. If you were running your WordCount programs in a large cluster or cloud
environment, and one of the nodes you were running on died mid computation,
how would your MapReduce and Spark programs handle this?
b. Explain one concrete benefit you experienced when writing the Spark version of
WordCount compared to the MapReduce version.



> .

>

>

>

>

>




.








# **Answers**





In [6]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
!pip install pyspark
!pip install pandas
!pip install nltk
!pip install pyngrok

In [3]:
from pyspark.sql import SparkSession
from pyngrok import ngrok
import pandas as pd

In [ ]:
# ngrok.disconnect(ngrok_tunnel.public_url)
# spark.stop()

## Initialize SparkContext and set **Spark UI** access point

In [5]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Spark UI")\
        .config("spark.ui.port", "4050")\
        .getOrCreate()

NGROK_TOKEN = "2cMzS23dCc40jCub1EmWsC7lLYF_6SvSu354ptk7Ri79vwJH"
ngrok.set_auth_token(NGROK_TOKEN)

ngrok_tunnel = ngrok.connect(4050, bind_tls=True)
print("Spark UI:", ngrok_tunnel.public_url)

Spark UI: https://bc01-35-230-164-173.ngrok-free.app


In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=Spark UI>

# Answer
## Part 1 A: Basic Word Count for a single text file

## Read single book

In [3]:
readsinglebook = sc.textFile("Little_Women.txt")


### Load and broadcast stopwords

In [9]:
import re
stopwords_path = "stopwords.txt"
with open(stopwords_path, 'r') as file:
    stopwords = set(file.read().split())
bcast = sc.broadcast(stopwords)


### Remove panctuation

In [10]:
def rmsplitpunon(line):
    line = re.sub(r'[^a-zA-Z\s]', '', line)
    return line.split()


### Perform Word Count

###### - Word count with filtering stopwords, and also removing punctuation, and numbers


Based on the assignment instractun in addition to basic word counting my code also do the following:
1. It must be case insensitive (see lower() in Python)
2. It must ignore all punctuation (see, for example, translate() in Python)
3. It must ignore stop words (see filter() in Spark)
4. The output must be sorted by count in descending order (see sortBy() in Spark)

In [11]:
wordcountsingle = readsinglebook.flatMap(lambda line: rmsplitpunon(line)) \
                                  .filter(lambda word: word.lower() not in bcast.value) \
                                  .map(lambda word: (word.lower(), 1)) \
                                  .reduceByKey(lambda x, y: x + y) \
                                  .sortBy(lambda x: x[1], ascending=False)



### Saving in Default  RDD Format and also as a text

In [14]:

savedir = "/outputs/wordcountsinglebook"

wordcountsingle.saveAsTextFile(savedir)

temp_output_dir = "/output/wordcountsinglebook"
wordcountsingle.coalesce(1).saveAsTextFile(temp_output_dir)
!mv $(find /content/wordcountsinglebook -type f -name 'part-00000') '/outputs/wordcountsinglebook.txt'
!rm -rf /content/wordcountsinglebook

## Display Top Words

In [15]:
top25 = wordcountsingle.take(25)
dfcountsingle = pd.DataFrame(top25, columns=['Word', 'Count Without Stopwords'])
dfcountsingle.index = range(1, len(dfcountsingle) + 1)
dfcountsingle

,Word,Count Without Stopwords
1,jo,841
2,meg,585
3,amy,385
4,dont,375
5,laurie,349
6,beth,315
7,mother,253
8,march,250
9,good,241
10,girls,217


## Lets compere without stopword and with stop word

In [16]:
withsword = readsinglebook.flatMap(lambda line: rmsplitpunon(line)) \
                                     .map(lambda word: (word.lower(), 1)) \
                                     .reduceByKey(lambda x, y: x + y) \
                                     .sortBy(lambda x: x[1], ascending=False)


withoutsword = readsinglebook.flatMap(lambda line: rmsplitpunon(line)) \
                                  .filter(lambda word: word.lower() not in bcast.value) \
                                  .map(lambda word: (word.lower(), 1)) \
                                  .reduceByKey(lambda x, y: x + y) \
                                  .sortBy(lambda x: x[1], ascending=False)

#### Comparison  of with stop word and without stop word

In [17]:
top25without = withoutsword.take(25)
top25with = withsword.take(25)


dfwithout = pd.DataFrame(top25without, columns=['Word', 'Count Without Stopwords'])
dfwith = pd.DataFrame(top25with, columns=['Word', 'Count With Stopwords'])

dfwithout.index = range(1, len(dfwithout) + 1)
dfwith.index = range(1, len(dfwith) + 1)

comparisondf = pd.concat([dfwithout, dfwith], axis=1)
comparisondf

,Word,Count Without Stopwords,Word,Count With Stopwords
1,jo,841,jo,841
2,meg,585,meg,585
3,amy,385,amy,385
4,dont,375,dont,375
5,laurie,349,laurie,349
6,beth,315,beth,315
7,mother,253,mother,253
8,march,250,march,250
9,good,241,good,241
10,girls,217,girls,217


## Save the Word Count Results

In [19]:
dir1 = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/compered"

outwithout1 = f"{dir1}/resultswithoutstopwords"
outwith1 = f"{dir1}/resultswithstopwords"

withoutsword.saveAsTextFile(outwithout1)
withsword.saveAsTextFile(outwith1)

### Read Back the Results

In [20]:
with open('/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/resultswithoutstopwords/part-00000', 'r') as file:
    lines = file.readlines()
top_25 = lines[:25]
for line in top_25:
    word, count = line.strip("()\n'").split(', ')
    word = word.strip("'")
    print(f"Word - {word}           -        Count - {count}")

Word - jo           -        Count - 841
Word - meg           -        Count - 585
Word - amy           -        Count - 385
Word - dont           -        Count - 375
Word - laurie           -        Count - 349
Word - beth           -        Count - 315
Word - mother           -        Count - 253
Word - march           -        Count - 250
Word - good           -        Count - 241
Word - girls           -        Count - 217
Word - time           -        Count - 195
Word - asked           -        Count - 189
Word - young           -        Count - 173
Word - face           -        Count - 170
Word - ill           -        Count - 165
Word - dear           -        Count - 162
Word - day           -        Count - 156
Word - illustration           -        Count - 143
Word - looked           -        Count - 137
Word - eyes           -        Count - 131
Word - cried           -        Count - 128
Word - great           -        Count - 124
Word - head           -        Count - 1

In [21]:
import pandas as pd

dir2 = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs"

withoutdir = f"{dir2}/resultswithoutstopwords"
withdir = f"{dir2}/resultswithstopwords"

import pandas as pd
import ast

def loadandtopd(dir):

    results = []

    with open(f"{dir}/part-00000", 'r') as file:
        for line in file:

            try:
                word, count = ast.literal_eval(line)
                results.append((word, count))
            except ValueError:
                print(f"Skipping line: {line}")

    df = pd.DataFrame(results, columns=['Word', 'Count'])

    return df

dfwithoutstopwords = loadandtopd(withoutdir)
dfwithstopwords = loadandtopd(withdir)

dfwithoutstopwords = dfwithoutstopwords.head(25)
dfwithstopwordstop = dfwithstopwords.head(25)

finaldf = pd.concat([dfwithstopwordstop.reset_index(drop=True),
                             dfwithoutstopwords.reset_index(drop=True)],
                            axis=1)

finaldf.columns = ['Word With Stopwords', 'occurance ', 'Word Without Stopwords', 'occurance']

finaldf

,Word With Stopwords,occurance,Word Without Stopwords,occurance
0,and,5095,jo,841
1,the,4808,meg,585
2,to,3125,amy,385
3,a,2833,dont,375
4,her,2235,laurie,349
5,of,2117,beth,315
6,i,1886,mother,253
7,in,1505,march,250
8,she,1446,good,241
9,it,1444,girls,217


## Part 1 B: Basic Word Count from multiple text file(books)

## From 10 Books

the above is for a single book now lets do it for 10 books

### Load/Read books

In [ ]:
bookspath = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/books/*"
read10books = sc.textFile(bookspath)

### Load and broadcast stopwords

In [ ]:
# Load and broadcast stopwords
stopwordspath = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/stopwords.txt"
with open(stopwordspath, 'r') as file:
    stopwords = set(file.read().split())
bcaststopwords = sc.broadcast(stopwords)

### Remove panctuation
and split

In [ ]:

def rmandsplitpanc(line):
    line = re.sub(r'[^a-zA-Z\s]', '', line)
    return line.split()


## Perform Word Count
- Word count with filtering stopwords, and also removing punctuation, and numbers

In [ ]:
word_counts_with_10books = read10books.flatMap(lambda line: rmandsplitpanc(line)) \
                                  .filter(lambda word: word.lower() not in bcaststopwords.value) \
                                  .map(lambda word: (word.lower(), 1)) \
                                  .reduceByKey(lambda x, y: x + y) \
                                  .sortBy(lambda x: x[1], ascending=False)


### Saving in Default RDD Format and also as a text

In [ ]:
# as RDD format
savedir10 = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/word_counts_with_10books"

word_counts_with_10books.saveAsTextFile(savedir10)


# as a text file
outdir = "/content/word_counts_with_10books"
wordcountsingle.coalesce(1).saveAsTextFile(outdir)
!mv $(find /content/word_counts_with_10books -type f -name 'part-00000') '/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/word_counts_with_10books.txt'
!rm -rf /content/word_counts_with_10books

### Display Top Words

In [ ]:
top20word=word_counts_with_10books.take(25)
top20wordDF = pd.DataFrame(top20word, columns=['Word', '   Count Without Stopwords from all 10 books'])

top20wordDF.index = range(1, len(top20wordDF) + 1)
top20wordDF

,Word,Count Without Stopwords from all 10 books
1,government,2723
2,united,2559
3,time,2488
4,great,2161
5,volume,1995
6,people,1610
7,general,1596
8,men,1579
9,man,1576
10,good,1395


## Lets compere without stopword and with stop word

In [ ]:
word_counts_with_stopwords = readf.flatMap(lambda line: rmandsplitpanc(line)) \
                                     .map(lambda word: (word.lower(), 1)) \
                                     .reduceByKey(lambda x, y: x + y) \
                                     .sortBy(lambda x: x[1], ascending=False)

word_counts_without_stopwords = readf.flatMap(lambda line: rmandsplitpanc(line)) \
                                  .filter(lambda word: word.lower() not in bcaststopwords.value) \
                                  .map(lambda word: (word.lower(), 1)) \
                                  .reduceByKey(lambda x, y: x + y) \
                                  .sortBy(lambda x: x[1], ascending=False)



## Display comparison  of with stop word and without stop word

In [ ]:
top_25_without = word_counts_without_stopwords.take(25)
top_25_with = word_counts_with_stopwords.take(25)


df_without_stopwords = pd.DataFrame(top_25_without, columns=['Word', 'Count Without Stopwords'])
df_with_stopwords = pd.DataFrame(top_25_with, columns=['Word', 'Count With Stopwords'])

df_without_stopwords.index = range(1, len(df_without_stopwords) + 1)
df_with_stopwords.index = range(1, len(df_with_stopwords) + 1)

comparison_df = pd.concat([df_without_stopwords, df_with_stopwords], axis=1)

comparison_df

## Save the Word Count Results

In [ ]:
# Directories to save the results
output_dir_without_stopwords = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/without_stopwords_from10books"
output_dir_with_stopwords = "/content/drive/MyDrive/MintesnotF-A5Spark-assignment/outputs/with_stopwords_from10books"

# Save the results
word_counts_without_stopwords.saveAsTextFile(output_dir_without_stopwords)
word_counts_with_stopwords.saveAsTextFile(output_dir_with_stopwords)

### Read Back the Results

In [ ]:
# To display top 25 words directly
top_25 = word_counts.take(25)
for word, count in top_25:
    print(f"Word - {word}           -        Count - {count}")

Word - the           -        Count - 129254
Word - of           -        Count - 80509
Word - and           -        Count - 52749
Word - to           -        Count - 49736
Word - in           -        Count - 34897
Word - a           -        Count - 33372
Word - that           -        Count - 19722
Word - is           -        Count - 15733
Word - as           -        Count - 15378
Word - was           -        Count - 14728
Word - with           -        Count - 13839
Word - for           -        Count - 13581
Word - it           -        Count - 13575
Word - be           -        Count - 13494
Word - by           -        Count - 12550
Word - i           -        Count - 11243
Word - which           -        Count - 11146
Word - on           -        Count - 10101
Word - he           -        Count - 9910
Word - his           -        Count - 9540
Word - at           -        Count - 9527
Word - not           -        Count - 9462
Word - this           -        Count - 9265
Wo